<a href="https://colab.research.google.com/github/cheikh87/baba/blob/master/modele_forest_perf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import files
uploaded = files.upload()

Saving data_finale_vanad.csv to data_finale_vanad (1).csv


KeyboardInterrupt: ignored

In [1]:
# Importation du jeu de données avec pandas
import pandas as pd# lire et affichage des 5 premières lignes
features = pd.read_csv('data_finale_vanad.csv')
features.head(5)

,LES,arrivTime,answerTime,qT,DureDeService,TempsAttente,somQo,agentNumber,Unnamed: 8
0,30560.0,4463.0,4463.0,4,201.0,0.0,0,6975.0,NaN
1,30172.0,6435.0,6435.0,4,373.0,0.0,0,6985.0,NaN
2,30560.0,3637.0,3638.0,0,314.0,1.0,0,6975.0,NaN
3,30175.0,4123.0,4124.0,1,21.0,1.0,0,7531.0,NaN
4,30175.0,4148.0,4149.0,2,226.0,1.0,0,6985.0,NaN


In [2]:
print('The shape of our features is:', features.shape)

The shape of our features is: (671162, 9)


In [3]:
features= features[features["qT"].astype(int)!=0]
features=features.fillna(value='0')


In [4]:
features.describe()

,LES,arrivTime,answerTime,qT,DureDeService,TempsAttente,somQo,agentNumber
count,655218.000000,655218.000000,655218.000000,655218.000000,655218.000000,655218.000000,655218.0,655218.000000
mean,30267.304862,16773.299824,16814.398574,135.527997,334.074993,41.098750,0.0,6619.955111
std,167.410334,9006.643611,9015.545120,253.273594,258.759075,82.227429,0.0,3307.891583
min,30066.000000,28.000000,29.000000,1.000000,-13027.000000,0.000000,0.0,1038.000000
25%,30175.000000,9245.000000,9286.000000,14.000000,159.000000,1.000000,0.0,1573.000000
50%,30175.000000,16110.500000,16164.000000,38.000000,276.000000,1.000000,0.0,8076.000000
75%,30241.000000,23807.750000,23849.000000,114.000000,440.000000,44.000000,0.0,9423.000000
max,30764.000000,44616.000000,44717.000000,1720.000000,5873.000000,1495.000000,0.0,9828.000000


In [6]:
# One-hot encode the data using pandas get_dummies
features = pd.get_dummies(features)# Display the first 5 rows of the last 12 columns
features.iloc[:,3:].head(8)

,qT,DureDeService,TempsAttente,somQo,agentNumber,Unnamed: 8_0
0,4,201.0,0.0,0,6975.0,1
1,4,373.0,0.0,0,6985.0,1
3,1,21.0,1.0,0,7531.0,1
4,2,226.0,1.0,0,6985.0,1
5,3,201.0,1.0,0,6975.0,1
7,1,226.0,1.0,0,6975.0,1
8,2,83.0,1.0,0,6985.0,1
9,3,136.0,1.0,0,6975.0,1


In [8]:
# conversion en tableau avec numpay
import numpy as np# On veut prédire les temps d'attente on crée le champ comme label et on le garde
labels = np.array(features['TempsAttente'])#
# axis 1 refers to the columns
features= features.drop('TempsAttente', axis = 1)# On supprime TempsAttente du jeu de données
feature_list = list(features.columns)# on crée une liste d'objet
#On transforme en tableau d'objet
features = np.array(features)

In [9]:
# Utilisation de Skicit-learn pour la séparation du jeu de donnnées en données entrainement et test
from sklearn.model_selection import train_test_split
# Séparation du jeu de données
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.5, random_state = 42)

In [10]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (163804, 8)
Training Labels Shape: (163804,)
Testing Features Shape: (491414, 8)
Testing Labels Shape: (491414,)


In [11]:
# The baseline predictions are the historical averages
baseline_preds = test_features[:, feature_list.index('qT')]# Baseline errors, and display average baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))


Average baseline error:  106.75


In [12]:
# Importation du module Randoms
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [13]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 19.44 degrees.


In [14]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: -inf %.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot# Pull out one tree from the forest
tree = rf.estimators_[5]# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot# Pull out one tree from the forest
tree = rf.estimators_[5]# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')# Write graph to a png file
graph.write_png('tree.png')